In [1]:
import os
from lib.AirtableLib import AirtableData

In [2]:
personal_base_id = 'appYknCML4IMHe9DP'
personal_table_id = 'tblQcjtQ7kB3dJx1m'
airtable_api_key = os.environ.get('AIRTABLE_API_KEY')

In [3]:
personal_airtable = AirtableData(personal_base_id, personal_table_id, airtable_api_key)
personal_data = personal_airtable.get_data()
print(personal_data)

           Date Score                                        Description  \
0    2023-02-19    -2  I was mainly just sick and tired still. AI cla...   
1    2023-03-25     1  This looks like a busy day, but in reality I j...   
2    2023-04-09     0  Today was my last day of AI class and wow does...   
3    2023-10-22     0  Woke up early for a dog walk before football w...   
4    2023-04-20     0  Solid day of bet-GPT work getting logins to wo...   
..          ...   ...                                                ...   
447  2024-01-17     0  I work up and got right to work. My team worki...   
448  2024-01-06     1  After a long week of Q5 work Maggie still had ...   
449  2023-09-20    -1  Basically the same as the last few days with a...   
450  2023-06-07     0  Work on the rcmOS project is improving, I'm ac...   
451  2023-07-05     0  Morning walk was a quick one so I made sure to...   

                                                Events       Location  \
0    [Work - A

## Oura Columns Work

In [4]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from typing import List, Dict, Any, Union

#### Testing Oura API endpoints

In [7]:
base_url = 'https://api.ouraring.com/v2/usercollection/'
access_token = os.environ.get('OURA_TOKEN')
endpoints = ['daily_activity','daily_readiness', 'daily_sleep', 'daily_spo2', 'daily_stress']
start_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

In [8]:
def test_oura_endpoint(endpoint, start_date, end_date):
    url = base_url + endpoint + '?start_date=' + start_date + '&end_date=' + end_date
    r = requests.get(url, headers={'Authorization': 'Bearer ' + access_token})
    return r.json()

In [9]:
# Test Each Endpoint
for endpoint in endpoints:
    data = test_oura_endpoint(endpoint, start_date, end_date)["data"]
    print(endpoint)
    print(f'Records: {len(data)}')
    print(f'Days: {[x["day"] for x in data]}')
    print(data)
    print()

daily_activity
Records: 1
Days: ['2024-01-26']
[{'id': 'c2342793-05f0-490a-8263-86ccbf92641d', 'class_5_min': '111111111111111111111111111111111111111112244342223344334443444422332222222322222322222223223322333222223333233112222222222322223222222222222222322332222233334455555430003334333332333333332233223322333222222233233233222222232222222112234332221111222223211111111111111111111111111111111111', 'score': 81, 'active_calories': 1246, 'average_met_minutes': 2.0, 'contributors': {'meet_daily_targets': 43, 'move_every_hour': 100, 'recovery_time': 100, 'stay_active': 72, 'training_frequency': 100, 'training_volume': 100}, 'equivalent_walking_distance': 20600, 'high_activity_met_minutes': 347, 'high_activity_time': 1860, 'inactivity_alerts': 0, 'low_activity_met_minutes': 162, 'low_activity_time': 17460, 'medium_activity_met_minutes': 514, 'medium_activity_time': 5760, 'met': {'interval': 60.0, 'items': [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 

In [25]:
activity_columns = ['day', 'score', 'active_calories', 'contributors__meet_daily_targets', 'contributors__move_every_hour',
                    'contributors__recovery_time', 'contributors__stay_active', 'contributors__training_frequency', 
                    'contributors__training_volume', 'steps', 'target_calories', 'target_meters', 'total_calories', 
                    'day', 'high_activity_met_minutes', 'high_activity_time', 'medium_activity_met_minutes',
                    'medium_activity_time', 'low_activity_met_minutes', 'low_activity_time', 'sedentary_met_minutes',
                    'sedentary_time', 'resting_time', 'equivalent_walking_distance', 'inactivity_alerts']

readiness_columns = ['day', 'score', 'contributors__activity_balance', 'contributors__body_temperature', 'contributors__hrv_balance',
                    'contributors__previous_day_activity', 'contributors__previous_night', 'contributors__recovery_index', 
                    'contributors__resting_heart_rate', 'contributors__sleep_balance', 'temperature_deviation', 'temperature_trend_deviation']

sleep_columns = ['day', 'score', 'contributors__deep_sleep', 'contributors__efficiency', 'contributors__latency', 'contributors__rem_sleep',
                'contributors__restfulness', 'contributors__timing', 'contributors__total_sleep']

spo2_columns = ['day', 'spo2_percentage__average']

stress_columns = ['day', 'stress_high', 'recovery_high', 'day_summary']

oura_columns = [activity_columns, readiness_columns, sleep_columns, spo2_columns, stress_columns]

In [26]:
oura_columns_with_enpoints = dict(zip(endpoints, oura_columns))
oura_columns_with_enpoints

{'daily_activity': ['day',
  'score',
  'active_calories',
  'contributors__meet_daily_targets',
  'contributors__move_every_hour',
  'contributors__recovery_time',
  'contributors__stay_active',
  'contributors__training_frequency',
  'contributors__training_volume',
  'steps',
  'target_calories',
  'target_meters',
  'total_calories',
  'day',
  'high_activity_met_minutes',
  'high_activity_time',
  'medium_activity_met_minutes',
  'medium_activity_time',
  'low_activity_met_minutes',
  'low_activity_time',
  'sedentary_met_minutes',
  'sedentary_time',
  'resting_time',
  'equivalent_walking_distance',
  'inactivity_alerts'],
 'daily_readiness': ['day',
  'score',
  'contributors__activity_balance',
  'contributors__body_temperature',
  'contributors__hrv_balance',
  'contributors__previous_day_activity',
  'contributors__previous_night',
  'contributors__recovery_index',
  'contributors__resting_heart_rate',
  'contributors__sleep_balance',
  'temperature_deviation',
  'temperatur

In [43]:
def read_oura_endpoint(endpoint: str, start_date: str, end_date: str, columns: List[str]) -> pd.DataFrame:
    # Retrieve data from the endpoint
    json_data = test_oura_endpoint(endpoint, start_date, end_date)["data"]
    print(json_data)

    # Create a dictionary of the data
    df_dict_array = []
    if len(json_data) > 0:
        try:
            for day in json_data:
                df_dict = {}
                for col in columns:
                    if '__' in col:
                        parent = col.split('__')[0]
                        child = col.split('__')[1]
                        df_dict[col] = day[parent][child]
                    else:
                        df_dict[col] = day[col]
                df_dict_array.append(df_dict)
        except TypeError as e:
            if 'NoneType' in str(e):
                pass
            else:
                raise e
            
    df = pd.DataFrame(df_dict_array).drop_duplicates()

    # Convert the date column to a datetime object
    df['day'] = pd.to_datetime(df['day']).dt.date

    # Set the date column as the index
    df = df.set_index('day')

    # Rename the columns
    # Remove the 'contributors__' prefix
    # Add the endpoint name as a prefix (excluding the daily part)
    # Change '_' to ' '
    # Uppercase the first letter of each word
    df = df.rename(columns=lambda x: x.replace('contributors__', '').replace('_', ' ').title())
    df = df.add_prefix(endpoint.replace('daily_', '').title() + ' ')

    return df

In [44]:
start_date = '2024-01-11'
end_date = datetime.now().strftime('%Y-%m-%d')

df_dict = {}

for endpoint in endpoints:
    df_dict[endpoint] = read_oura_endpoint(endpoint, start_date, end_date, oura_columns_with_enpoints[endpoint])

df_dict

[{'id': 'c9695022-9689-45e3-83c3-d5d6ec7306fe', 'class_5_min': '000000000000000000000000000000000000000000000000000000000000000000000000000000000000000223212222222222332222222222222212222222222222322222222222322344333433334433433213333332224343333323322233332222222222222223223333322222234223223222211111111111111111111111111111111111111111111111111111', 'score': 92, 'active_calories': 393, 'average_met_minutes': 1.34375, 'contributors': {'meet_daily_targets': 78, 'move_every_hour': 95, 'recovery_time': 100, 'stay_active': 86, 'training_frequency': 100, 'training_volume': 100}, 'equivalent_walking_distance': 7437, 'high_activity_met_minutes': 7, 'high_activity_time': 60, 'inactivity_alerts': 1, 'low_activity_met_minutes': 175, 'low_activity_time': 13860, 'medium_activity_met_minutes': 133, 'medium_activity_time': 2400, 'met': {'interval': 60.0, 'items': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 

{'daily_activity':             Activity Score  Activity Active Calories  \
 day                                                    
 2024-01-11              92                       393   
 2024-01-12              93                       803   
 2024-01-13              95                       569   
 2024-01-14              98                      1918   
 2024-01-15              97                      1122   
 2024-01-16              94                       930   
 2024-01-17              91                       338   
 2024-01-18              96                       969   
 2024-01-19              98                       973   
 2024-01-20              94                       603   
 2024-01-21              97                       485   
 2024-01-22              88                       857   
 2024-01-23              87                       385   
 2024-01-24              83                       491   
 2024-01-25              80                       468   
 2024-01-26  

In [45]:
for df_key in df_dict.keys():
    print(df_key)
    print(df_dict[df_key].describe())
    print()

daily_activity
       Activity Score  Activity Active Calories  Activity Meet Daily Targets  \
count           16.00                 16.000000                     16.00000   
mean            91.50                784.375000                     83.25000   
std              6.00                414.381768                     20.53777   
min             80.00                338.000000                     43.00000   
25%             87.75                480.750000                     73.50000   
50%             93.50                703.000000                     95.00000   
75%             96.25                970.000000                     96.25000   
max             98.00               1918.000000                    100.00000   

       Activity Move Every Hour  Activity Recovery Time  Activity Stay Active  \
count                    16.000                    16.0             16.000000   
mean                     98.125                   100.0             73.062500   
std                  